# From Text to a Knowledge Graph

Based on article: *[From Text to a Knowledge Graph. Hands-on](https://medium.com/@hugojm/from-text-to-a-knowledge-graph-hands-on-dd68e9d42939)*

Libraries: 
* [DBpedia Spotlight for SpaCy](https://spacy.io/universe/project/spacy-dbpedia-spotlight): **pip install spacy_dbpedia_spotlight**
* [Getting started with RDFLib](https://rdflib.readthedocs.io/en/stable/gettingstarted.html): **conda install -c conda-forge rdflib**

In [1]:
import spacy_dbpedia_spotlight
from spacy.matcher import Matcher

UnsupportedOperation: fileno

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('dbpedia_spotlight')

In [ ]:
doc = nlp("Joe Biden married Jill Tracy Jacobs, who was educated at the University of Delaware")

In [ ]:
print([token.text for token in doc], sep=' ')